# Loading Streak camera data

In [23]:
%matplotlib qt
import os, glob
import numpy as np
import lumispy as lum
import hyperspy.api as hs
import matplotlib.pyplot as plt
import addcopyfighandler

In [73]:
# PARAMS
root = os.path.abspath('G:\My Drive\PhD')
session = r'projects\external_measurements\20211118_CL_Mn_doped_NCs_Sascha'
stacks_folder = 'STREAK'
file_end = '*.dac'

files = glob.glob(os.path.join(root, session, stacks_folder, file_end))
files

['G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\STREAK\\STREAK101_A_130k_478nm_timerange4_10nA_laser5p5_dwellt10ms_z30.dac',
 'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\STREAK\\STREAK102_A_147k_478nm_timerange4_10nA_laser5p5_dwellt10ms_z0.dac',
 'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\STREAK\\STREAK103_F_120k_478nm_timerange4_10nA_laser5p5_dwellt10ms_z0.dac',
 'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\STREAK\\STREAK104_F_130k_478nm_timerange4_10nA_laser5p5_dwellt10ms_z30.dac',
 'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\STREAK\\STREAK1_B_200k_474p5nm_timerange4_10nA_laser5p5_dwellt10ms.dac',
 'G:\\My Drive\\PhD\\projects\\external_measurements\\20211118_CL_Mn_doped_NCs_Sascha\\STREAK\\STREAK2_E_200k_474p5nm_timerange4_10nA_laser5p5_dwellt10ms.dac',
 'G:\\My Drive\\PhD\\proje

Get the metadata from the `.tif` file

In [63]:
def get_md_dictionary_from_hs_tif_file(hs_tif_file):
    md_dict = {}
    md = hs_tif_file.original_metadata.ImageDescription.split('\r\n')
    for m in md:
        l = m.split(",")
        sub_dict = {}
        for i in l:
            try:
                k,v = i.split('=')
            except Exception:
                k = i
                v = False
            sub_dict[k] = v
        category_name = l[0].replace('[', '').replace(']','')
        md_dict[category_name] = sub_dict
    return md_dict

Load the `.dac` data

In [139]:
# Tidy file
def get_axes_units(np_data):
    units = np_data[0,0]
    units_y, units_x = units.split('|')
    return units_x, units_y

def get_x_y_axes(np_data):
    x_axis = np_data[:1,1:].flatten()
    y_axis = np_data[1:,:1].flatten()
    return x_axis.astype(float), y_axis.astype(float)

def get_data_matrix(np_data):
    data = np_data[1:,1:]
    return data.astype(int)

def load_dac_file(dac_file_path, metadata=None,
                  original_metadata=None, to_linear_axes=False):

    with open(dac_file_path, 'r') as reader:
        dat = reader.readlines()
        dat = np.array([d.split('\t') for d in dat])
        dat = np.char.strip(dat, '\n')

    units = get_axes_units(dat)
    axes = get_x_y_axes(dat)
    data = get_data_matrix(dat)

    if to_linear_axes:
        from hyperspy.axes import UniformDataAxis
        x_offset = axes[0].min()
        x_size = len(axes[0])
        x_scale = (axes[0].max() - x_offset) / x_size
        x_axis = UniformDataAxis(scale=x_scale, offset=x_offset, size=x_size,
                                 name='Energy', units=units[0], navigate=False)

        y_offset = axes[1].min()
        y_size = len(axes[1])
        y_scale = (axes[1].max() - y_offset) / y_size
        y_axis = UniformDataAxis(scale=y_scale, offset=y_offset, size=y_size,
                                 name='Time', units=units[1], navigate=False)

        axes_dict = (y_axis.get_axis_dictionary(), x_axis.get_axis_dictionary())

    else:
        from hyperspy.axes import DataAxis
        x_axis = DataAxis(axis=axes[0], name='Energy', units=units[0], navigate=False)
        y_axis = DataAxis(axis=axes[1], name='Time', units=units[1], navigate=False)
        axes_dict = (y_axis.get_axis_dictionary(), x_axis.get_axis_dictionary())

    s = lum.signals.luminescence_transient.LumiTransient(data, axes=axes_dict)
    #s = hs.signals.Signal2D(data, axes=axes_dict)

    if metadata is not None:
        s.metadata = metadata
    if original_metadata is not None:
        s.original_metadata = original_metadata
    try:
        s.metadata.General.title = s.metadata.General.original_filename.split('.')[0]
    except Exception:
        print('No file name found.')

    return s

In [157]:
force_linear_axes = True

streaks = []
for f in files[:]:
    f_name = os.path.basename(f).split('.')[0]
    dir_name = os.path.dirname(f)
    f_md = os.path.join(dir_name, f"{f_name}.tif")
    f_md = hs.load(f_md)
    md = get_md_dictionary_from_hs_tif_file(f_md)
    streak = load_dac_file(f, metadata=f_md.metadata, #original_metadata=md,
                           to_linear_axes=force_linear_axes)
    #streak.save(os.path.join(dir_name, f"{f_name}.hspy"))
    streaks.append(streak)

streaks

[<LumiTransient, title: STREAK101_A_130k_478nm_timerange4_10nA_laser5p5_dwellt10ms_z30, dimensions: (|672, 508)>,
 <LumiTransient, title: STREAK102_A_147k_478nm_timerange4_10nA_laser5p5_dwellt10ms_z0, dimensions: (|672, 508)>,
 <LumiTransient, title: STREAK103_F_120k_478nm_timerange4_10nA_laser5p5_dwellt10ms_z0, dimensions: (|672, 508)>,
 <LumiTransient, title: STREAK104_F_130k_478nm_timerange4_10nA_laser5p5_dwellt10ms_z30, dimensions: (|672, 508)>,
 <LumiTransient, title: STREAK1_B_200k_474p5nm_timerange4_10nA_laser5p5_dwellt10ms, dimensions: (|672, 508)>,
 <LumiTransient, title: STREAK2_E_200k_474p5nm_timerange4_10nA_laser5p5_dwellt10ms, dimensions: (|672, 508)>,
 <LumiTransient, title: STREAK3_E_220k_475nm_timerange4_10nA_laser5p5_dwellt10ms_sameSpot, dimensions: (|672, 508)>,
 <LumiTransient, title: STREAK4_B_340k_475nm_timerange4_10nA_laser5p5_dwellt10ms_sameSpot, dimensions: (|672, 508)>]

In [141]:
streaks[1].plot(aspect="auto", cmap='viridis')

#  Plot the results

In [110]:
def plot_contour_of_streak(transient_lumi, nbins=15, cmap='viridis', figsize=(6,4),
                           annotation=None):


    scan_name = transient_lumi.metadata.General.title

    axes = transient_lumi.axes_manager.signal_axes
    x = axes[0].axis
    t = axes[1].axis
    dat = transient_lumi.data

    vmax = np.quantile(dat, q=0.99)
    X, Y = np.meshgrid(x,t)

    from matplotlib.colors import BoundaryNorm
    from matplotlib.ticker import MaxNLocator
    levels = MaxNLocator(nbins=nbins).tick_values(0, vmax)
    cmap = plt.get_cmap(cmap)
    norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

    f, ax = plt.subplots(figsize=figsize)
    im = ax.pcolormesh(X, Y, dat, cmap=cmap, norm=norm)
    f.colorbar(im, ax=ax)
    ax.set_xlabel(f"{axes[0].name} ({axes[0].units})")
    ax.set_ylabel(f"{axes[1].name} ({axes[1].units})")
    ax.invert_yaxis()
    f.suptitle(scan_name)
    if annotation is not None:
        ax.annotate(annotation,
            xy=(0.95,0.05), xycoords='axes fraction',
            horizontalalignment='right', verticalalignment='bottom',
            bbox=dict(boxstyle="round", fc="white", alpha=0.8))
    plt.tight_layout()
    return f

In [111]:
i=1
fig = plot_contour_of_streak(streaks[i])

C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\ipykernel_launcher.py:22: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.


In [56]:
for s in streaks[:]:
    exp_t = s.original_metadata['Acquisition']['ExposureTime']
    reps = int(s.original_metadata['Acquisition']['NrExposure'])
    annotation = f"{exp_t} ({reps:,} reps)"
    fig = plot_contour_of_streak(s, annotation=annotation)
    name = f"{s.metadata.General.title}.png"
    path = os.path.join(root, session, stacks_folder, 'plots', name)
    plt.savefig(path)
    plt.close()

C:\Users\jf631\miniconda3\envs\lumispy\lib\site-packages\ipykernel_launcher.py:22: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.


# Streak camera interactive plotting (NOT WORKING FOR NON-LINEAR AXES)

Select vertical and horizontal integration windows to get time decay and energy distribution.

In [158]:
# Set file to plot

i = 0
im = streaks[i]

In [198]:
# Set default integration windows
energy_range = im.axes_manager.signal_axes[0].axis
time_range = im.axes_manager.signal_axes[1].axis

energy_roi_range = (np.quantile(energy_range, q=0.25),
                  time_range.min(),
                  np.quantile(energy_range, q=0.75),
                  time_range.max(),)

time_roi_range = (energy_range.min(),
            np.quantile(time_range, q=0.15),
            energy_range.max(),
            np.quantile(time_range, q=0.75),)

energy_roi_range

(452.3333952380952, 722.337, 499.3097857142857, 1998.4849429133858)

In [199]:
# Plot interactively
im.plot(aspect="auto", cmap='viridis')
energy_roi = hs.roi.RectangularROI(*energy_roi_range)
energy_roi_i = energy_roi.interactive(im, color="C0")
energy_line = hs.interactive(energy_roi_i.sum,
                             event=energy_roi.events.changed,
                             axis=1,
                             recompute_out_event=None,)

time_roi = hs.roi.RectangularROI(*time_roi_range)
time_roi_i = time_roi.interactive(im, color="C1")
time_line = hs.interactive(time_roi_i.sum,
                             event=time_roi.events.changed,
                             axis=0,
                             recompute_out_event=None,)

energy_line.plot(color='C0')
time_line.plot(color='C1')
